In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr import load_dataset, load_dataset_scheme, load_dataset_config, Dataset
from lib.ehr.interface import Patients, InterfaceConfig
from lib.ehr.concepts import DemographicVectorConfig, LeadingObservableConfig


In [3]:
import logging
logging.root.level = logging.INFO

In [4]:
tag = 'M4ICU'
PATH = f'{os.environ.get("HOME")}/GP/ehr-data/mimic4icu-cohort'
sample = 300
offset = 50
cache =  f'cached_inteface/patients_{tag}_S{sample}_O{offset}'
dataset_config = load_dataset_config(tag, 
                                     sample=sample,
                                     offset=offset,
                                     path=PATH)

In [5]:
cache

'cached_inteface/patients_M4ICU_S300_O50'

##### Possible Interface Scheme Configurations

In [6]:
import json
dataset_scheme = load_dataset_scheme(tag)
interface_schem_options = dataset_scheme.supported_target_scheme_options
print(json.dumps(interface_schem_options, sort_keys=True, indent=4))

{
    "dx": [
        "DxFlatCCS",
        "DxCCS",
        "DxICD9",
        "DxICD10"
    ],
    "ethnicity": [
        "MIMIC4Eth32",
        "MIMIC4Eth5"
    ],
    "gender": [
        "Gender"
    ],
    "int_input": [
        "MIMICInput",
        "MIMICInputGroups"
    ],
    "int_proc": [
        "MIMICProcedures",
        "MIMICProcedureGroups"
    ],
    "obs": [
        "MIMICObservables"
    ],
    "outcome": [
        "dx_flatccs_filter_v1",
        "dx_icd9_filter_v3_groups",
        "dx_flatccs_mlhc_groups",
        "dx_icd9_filter_v1",
        "dx_icd9_filter_v2_groups"
    ]
}


#### Leading Observable for Early Prediction Task

In [7]:
scheme_df = dataset_scheme.obs.as_dataframe()
display(scheme_df[scheme_df.desc.str.contains('aki')])

,code,desc
42,o42,aki_stage_smoothed


In [8]:

# Demographic vector attributes
demographic_vector_conf = DemographicVectorConfig(
    age=False, 
    gender=False, 
    ethnicity=False
)

# Leading 
leading_AKI = LeadingObservableConfig(leading_hours=[6, 12, 24, 48, 72],
                                      window_aggregate='max',
                                      scheme=dataset_scheme.obs,
                                      index=42)

In [9]:
interface_scheme = dataset_scheme.make_target_scheme_config(dx='DxICD9',
                                                            outcome='dx_icd9_filter_v3_groups',
                                                            ethnicity='MIMIC4Eth5')
interface_config = InterfaceConfig(scheme=interface_scheme,
                                   dataset_scheme=dataset_scheme,
                                   demographic_vector=demographic_vector_conf,
                                   leading_observable=leading_AKI,
                                   cache=cache)

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [10]:
from lib.ml import (InICENODE, InICENODEConfig, InpatientEmbeddingConfig,  SplitConfig,
                    InTrainer, TrainerConfig, TrainerReporting, OptimizerConfig, WarmupConfig, ReportingConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC, CodeLevelMetricConfig, MetricLevelsConfig,
                         LossMetricConfig, CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric, 
                         CodeGroupTopAlarmAccuracyConfig, LeadingObsTrends, 
                        AKISegmentedAdmissionMetric, AKISegmentedAdmissionConfig)
from lib.ml import Experiment, InpatientExperiment, ExperimentConfig, SplitConfig

import jax.random as jrandom

In [11]:
emb_dims = InpatientEmbeddingConfig(dx=30, inp=15, proc=15, 
                                        demo=0, 
                                        inp_proc_demo=10)
model_config = InICENODEConfig(mem=15, obs=25, lead=5, emb=emb_dims,
                              lead_predictor='mlp')
model_classname = InICENODE.__name__

In [12]:
trainer_config = TrainerConfig(optimizer=OptimizerConfig(opt='adam', lr=1e-3),
                          epochs=3,
                          batch_size=64,
                          dx_loss='balanced_focal_bce',
                          obs_loss='mse',
                          lead_loss='mse')

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)




In [13]:
dx_loss = ["softmax_bce", "balanced_focal_softmax_bce", "balanced_focal_bce",
          "allpairs_exp_rank", "allpairs_hard_rank", "allpairs_sigmoid_rank"]
lead_loss = ["mse", "mae", "rms", "softdtw(0.1)"]
obs_loss =  ["mse", "mae", "rms"]
                
metrics_conf = [
    (CodeAUC, CodeLevelMetricConfig(aggregate_level=True, code_level=True)),
    (AdmissionAUC, MetricLevelsConfig(admission=False, aggregate=True, subject_aggregate=False)),
    (CodeGroupTopAlarmAccuracy, CodeGroupTopAlarmAccuracyConfig(n_partitions=5, top_k_list=[3, 5, 10, 15, 20])),
    (LossMetric, LossMetricConfig(dx_loss=dx_loss, lead_loss=lead_loss, obs_loss=obs_loss)),
    (LeadingObsTrends, CodeLevelMetricConfig(aggregate_level=True, code_level=True)),
    (AKISegmentedAdmissionMetric, AKISegmentedAdmissionConfig())
]
metrics_conf = [m.export_module_class(c) for m, c in metrics_conf]

In [14]:
reporting_conf = ReportingConfig(output_dir='mlp_inicenode',
                                 console=True,
                                 model_stats=False,
                                 parameter_snapshots=True,
                                 config_json=True)

In [15]:
expt_config = ExperimentConfig(dataset=dataset_config,
                              interface=interface_config,
                              split=SplitConfig(train=0.8, val=0.1, test=0.1, balanced='admissions'),
                              trainer=trainer_config,
                              metrics=metrics_conf,
                              reporting=reporting_conf,
                              model=model_config,
                              model_classname=model_classname,
                              n_evals=100,
                              continue_training=True,
                              warmup=None,
                              reg_hyperparams=None)

In [16]:
experiment = InpatientExperiment(expt_config)

In [17]:
result = experiment.run()

INFO:root:Loading cached subjects.
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Interpreter CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:root:Continuing training from step 3
INFO:root:HPs: TrainerConfig(
  optimizer=OptimizerConfig(
    opt='adam',
    lr=0.001,
    decay_rate=None,
    reverse_schedule=False
  ),
  epochs=3,
  batch_size=64,
  dx_loss='balanced_focal_bce',
  obs_loss='mse',
  lead_loss='mse'
)


Loading to device:   0%|          | 0/29 [00:00<?, ?subject/s]

  0%|          | 0/3 [00:00<?, ?Epoch/s]

  0%|          | 0/13 [00:00<?, ?Batch/s]

Loading to device:   0%|          | 0/16 [00:00<?, ?subject/s]

...

/home/asem/GP/env/icenode-dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Loading to device:   0%|          | 0/22 [00:00<?, ?subject/s]

Embedding:   0%|          | 0/22 [00:00<?, ?subject/s]

  0%|          | 0.00/620.78 [00:00<?, ?longitudinal-days/s]

/home/asem/GP/env/icenode-dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/asem/GP/env/icenode-dev/lib/python3.10/zipfile.py:1519: UserWarning: Duplicate name: 'step0039.eqx'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [1]:
import pandas as pd

# df = pd.read_csv('/home/asem/GP/ehr-data/m4icu_out/mlp_mse_icenodelite/val_evals.csv.gz')
df = pd.read_csv('/home/asem/GP/ehr-data/m4icu_out/mlp_mse_icenode/val_evals.csv.gz')

# df = pd.read_csv('mlp_inicenode/val_evals.csv.gz')

In [2]:
auc1_cols = [c for c in df.columns if 'first_emergence_auc' in c]
auc2_cols = [c for c in df.columns if 'AKISegmentedAdmissionMetric.emergence_auc' in c]
auc3_cols = [c for c in df.columns if 'all_emergence_auc' in c]
loss_cols = [c for c in df.columns if 'Loss' in c]

In [4]:
df[auc1_cols]

,AKISegmentedAdmissionMetric.first_emergence_auc_1-72,AKISegmentedAdmissionMetric.first_emergence_auc_6-72,AKISegmentedAdmissionMetric.first_emergence_auc_12-72,AKISegmentedAdmissionMetric.first_emergence_auc_18-72,AKISegmentedAdmissionMetric.first_emergence_auc_24-72,AKISegmentedAdmissionMetric.first_emergence_auc_30-72,AKISegmentedAdmissionMetric.first_emergence_auc_36-72,AKISegmentedAdmissionMetric.first_emergence_auc_42-72,AKISegmentedAdmissionMetric.first_emergence_auc_48-72,AKISegmentedAdmissionMetric.first_emergence_auc_54-72,AKISegmentedAdmissionMetric.first_emergence_auc_60-72,AKISegmentedAdmissionMetric.first_emergence_auc_66-72
0,0.972458,0.964153,0.952149,0.941346,0.933123,0.928199,0.919412,0.908913,0.906658,0.901982,0.888069,0.86956


In [6]:
df['elapsed_time'] / 3600

0    0.245981
Name: elapsed_time, dtype: float64